## Haxby data set:
Haxby is a high-quality block-design fMRI dataset from a study on face & object representation in the human ventral temporal cortex (This cortex is involved in the high-level visual processing of complex stimuli). It consists of 6 subjects with 12 runs per subject. In this experiment during each run, the subjects passively viewed greyscale images of 8 object categories, grouped in 24s blocks separated by rest periods. Each image was shown for 500ms and was followed by a 1500ms inter-stimulus interval.

## Project Goal
For this project i am trying machine learning and deep learning methods to learning about barin decoding and predicting which object category the subject saw by analyzing the fMRI activity recorded masks of the ventral stream.

In [1]:
import nibabel as nib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

from nilearn.plotting import plot_anat, show, plot_stat_map, plot_matrix
from nilearn import datasets, plotting, image
from nilearn.image import mean_img, get_data 
from nilearn.input_data import NiftiMasker
from sklearn.model_selection import train_test_split, LeaveOneGroupOut, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression, RidgeClassifier,  RidgeClassifierCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_selection import SelectPercentile, f_classif, SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import tree

In [2]:
#%matplotlib inline
#%load_ext memory_profiler

## Dataset

In [3]:
# If we don't define which subject by default 2nd subject will be fetched.
haxby_ds = datasets.fetch_haxby(subjects=[4], fetch_stimuli=True)
# print(haxby_ds)

# To define which subject to be fetched:
# haxby_ds = datasets.fetch_haxby(subjects=[], fetch_stimuli=True)

len(haxby_ds.func)

1

In [4]:
# Look inside the data
haxby_ds.keys()

dict_keys(['anat', 'func', 'session_target', 'mask_vt', 'mask_face', 'mask_house', 'mask_face_little', 'mask_house_little', 'mask', 'description', 'stimuli'])

In [5]:
mask_file = haxby_ds.mask
labels = haxby_ds.session_target[0]
mask_vt_file = haxby_ds.mask_vt[0]
mask_face_file = haxby_ds.mask_face[0]

# 'func' is a list of filenames: one for each subject
func_file = haxby_ds.func[0]

# Load the behavioral data that I will predict
beh_label = pd.read_csv(haxby_ds.session_target[0], sep=" ")

# Extract tags indicating to which acquisition run a tag belongs
session = beh_label['chunks']

# Preparing the data (Load target information as string and give a numerical identifier to each)
stimuli_lable = beh_label['labels']

# Identify the resting state
nonrest_task_mask = (y != 'rest')

# Remove the resting state and find names of remaining active labels
categories = stimuli_lable[nonrest_task_mask].unique()
#session = session[nonrest_task_mask]

# Get the labels of the numerical conditions represented by the vector y
unique_conditions, order = np.unique(categories, return_index=True)

# Sort the conditions by the order of appearance
unique_conditions = unique_conditions[np.argsort(order)]

# Extract tags indicating to which acquisition run a tag belongs
session_labels = beh_label['chunks'][nonrest_task_mask]

In [6]:
# Print basic information on the dataset
print('Functional nifti images are located at: %s' % haxby_ds.func[0])
print('Mask nifti image (3D) is located at:  %s' % haxby_ds.mask)
print('First subject functional nifti images (4D) are at: %s' %func_file)  # 4D data

Functional nifti images are located at: /home/srastegarnia/nilearn_data/haxby2001/subj4/bold.nii.gz
Mask nifti image (3D) is located at:  /home/srastegarnia/nilearn_data/haxby2001/mask.nii.gz
First subject functional nifti images (4D) are at: /home/srastegarnia/nilearn_data/haxby2001/subj4/bold.nii.gz


In [7]:
# Checkout the confounds of the data
session_target = pd.read_csv(haxby_ds['session_target'][0], sep='\t')

session_target.head()

,labels chunks
0,rest 0
1,rest 0
2,rest 0
3,rest 0
4,rest 0


## Preparing the fMRI data (smooth and apply the mask)

In [8]:
# For decoding, standardizing is important, I am also smoothing the data

nifti_masker = NiftiMasker(mask_img=mask_file, standardize=True, sessions=session,  smoothing_fwhm=4,
                           memory="nilearn_cache", memory_level=1)

X = nifti_masker.fit_transform(func_file)


# Remove the resting state
#X = X[nonrest_task_mask]


## Plot Haxby masks

In [9]:
masker = NiftiMasker(mask_img=mask_vt_file, standardize=True)
fmri_masked = masker.fit_transform(func_file)

# Report depict the computed mask
# masker.generate_report()

In [10]:
# The variable “fmri_masked” is a numpy array
print(fmri_masked)

[[-2.6038213  -4.024481    0.82268804 ... -1.3077999  -0.76046395
  -0.80006576]
 [-2.6749918  -4.101693    1.2567352  ... -1.4760997  -0.8379458
  -0.6758426 ]
 [-3.1494625  -3.5612085   1.0739785  ... -1.1395003  -1.0962186
  -0.80006576]
 ...
 [-1.4176446  -0.7429663  -1.5303041  ... -0.992238   -0.5538457
  -0.9656967 ]
 [-1.038068   -1.6502086  -1.6902162  ... -0.5083763  -0.81211853
  -0.75865805]
 [-0.7296622  -0.91669357 -1.484615   ...  0.5645344   0.24679996
  -0.1789499 ]]


In [11]:
print(fmri_masked.shape)

(1452, 675)


## Converting the Mask to a Matrix

In [12]:
# load bold image into memory as a nibabel image
func = nib.load(func_file)

# load mask image into memory as a nibabel image
mask = nib.load(mask_file) 

# get the physical data of the mask (3D matrix of voxels)
mask_data = mask.get_data() 

print(func.shape)
print(mask.shape) 
print(len(mask_data[mask_data==1]))

(40, 64, 64, 1452)
(40, 64, 64)
39912


/home/srastegarnia/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  


In [13]:
# Create the masker object 
masker = NiftiMasker(mask_img=mask_file, standardize=True)

# Create a numpy matrix from the BOLD data, using the mask for the transformation
#%memit bold_masked = masker.fit_transform(bold_path)
func_masked = masker.fit_transform(func_file)

# View the dimensions of the matrix. The shape represents the number of time-stamps by the number of voxels in the mask. 
print(func_masked.shape)

(1452, 39912)


In [14]:
# Viewing the numerical values of the matrix
print(func_masked)

[[ 1.851505   -1.1370772  -1.029912   ... -0.97751945 -0.9890595
  -0.9778365 ]
 [ 1.6732124  -1.1370772  -1.029912   ... -0.97751945 -0.9890595
  -0.9778365 ]
 [-1.0308924  -1.1370772  -1.029912   ... -0.97751945  1.6580281
   1.1377147 ]
 ...
 [ 0.12800965  0.75440264  0.08345481 ... -0.97751945 -0.9890595
  -0.9778365 ]
 [ 0.6331721   0.8390958   0.32418278 ... -0.97751945 -0.9890595
  -0.9778365 ]
 [ 0.5143103   1.064944   -0.12718217 ... -0.97751945 -0.9890595
  -0.9778365 ]]


In [15]:
# Load the labels from a csv into an array using pandas
stimuli = pd.read_csv(labels, sep=' ')

In [16]:
# View the dimensions of the matrix
print(stimuli.shape)

# Viewing the values of the matrix
print(stimuli)

(1452, 2)
     labels  chunks
0      rest       0
1      rest       0
2      rest       0
3      rest       0
4      rest       0
...     ...     ...
1447   rest      11
1448   rest      11
1449   rest      11
1450   rest      11
1451   rest      11

[1452 rows x 2 columns]


In [17]:
targets = stimuli['labels']
print(targets)

0       rest
1       rest
2       rest
3       rest
4       rest
        ... 
1447    rest
1448    rest
1449    rest
1450    rest
1451    rest
Name: labels, Length: 1452, dtype: object


In [18]:
targets_mask = targets.isin(['face', 'cat'])
print(targets_mask)

0       False
1       False
2       False
3       False
4       False
        ...  
1447    False
1448    False
1449    False
1450    False
1451    False
Name: labels, Length: 1452, dtype: bool


In [19]:
func_masked = func_masked[targets_mask]
func_masked.shape

(216, 39912)

In [20]:
targets_masked = targets[targets_mask]

print(targets_masked.shape)
print(targets_masked)

(216,)
6       face
7       face
8       face
9       face
10      face
        ... 
1370     cat
1371     cat
1372     cat
1373     cat
1374     cat
Name: labels, Length: 216, dtype: object


# ML Models

## Different classifiers for decoding
In this section I am willing to compare the different classifiers on a visual object recognition decoding task.

### Support Vector Machines

In [21]:
#select data
X = masker.fit_transform(func_file)
y = beh_label['labels']

#shuffle the data and split the sample into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True)

#standarize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Support vector classifier
svm = SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

svm.fit(X_train, y_train)

#accuracy
score = svm.score(X_test, y_test)

#print classification report
svm = svm.predict(X_test)
report = classification_report(y_test, svm)
print(report)

print("Test score with L1 penalty: %.4f" % score)

              precision    recall  f1-score   support

      bottle       0.88      1.00      0.93         7
         cat       0.90      0.69      0.78        13
       chair       0.88      0.88      0.88         8
        face       0.92      0.86      0.89        14
       house       1.00      0.92      0.96        13
        rest       0.84      0.92      0.88        63
    scissors       0.89      0.89      0.89         9
scrambledpix       0.91      0.91      0.91        11
        shoe       1.00      0.75      0.86         8

    accuracy                           0.88       146
   macro avg       0.91      0.87      0.89       146
weighted avg       0.89      0.88      0.88       146

Test score with L1 penalty: 0.8836


In [22]:
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier

### Logistic Regression

In [23]:
#select data
X = masker.fit_transform(func_file)
y = beh_label['labels']

#shuffle the data and split the sample into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True)

#standarize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#multinomial logistic regression object using L1 penalty
logistic_50 = LogisticRegression(C=50., multi_class='multinomial', penalty='l1', solver='saga', tol=0.1)

#train model
logistic_50.fit(X_train, y_train)
sparsity = np.mean(logistic_50.coef_ == 0) * 100
score = logistic_50.score(X_test, y_test)

# print('Best C % .4f' % clf.C_)
print("Sparsity with L1 penalty: %.2f%%" % sparsity)
print("Test score with L1 penalty: %.4f" % score)

Sparsity with L1 penalty: 0.00%
Test score with L1 penalty: 0.8151


### Nearest Neighbours

In [24]:
#select data
X = masker.fit_transform(func_file)
y = beh_label['labels']

#shuffle the data and split the sample into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True)

#standarize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#kneighbors classifier object
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='kd_tree',
                           p=2, metric='minkowski', metric_params=None, n_jobs=None)

#fit model
knn.fit(X_train, y_train)

#response prediction
pred = knn.predict(X_test)

#accuracy
knn.score(X_test, y_test)

#print classification report
knn = knn.predict(X_test)
report = classification_report(y_test, knn)
print(report)

#evaluate accuracy
print("Accuracy on test set: %0.3f%%"%(accuracy_score(y_test, pred)*100))

              precision    recall  f1-score   support

      bottle       0.31      0.80      0.44         5
         cat       0.64      0.82      0.72        11
       chair       0.80      0.92      0.86        13
        face       0.88      1.00      0.93        14
       house       0.80      0.62      0.70        13
        rest       0.93      0.64      0.76        64
    scissors       0.80      0.80      0.80        10
scrambledpix       0.42      1.00      0.59         5
        shoe       0.75      0.82      0.78        11

    accuracy                           0.75       146
   macro avg       0.70      0.82      0.73       146
weighted avg       0.82      0.75      0.76       146

Accuracy on test set: 75.342%


## Decision trees

In [25]:
#select data
X = masker.fit_transform(func_file)
y = beh_label['labels']

#shuffle the data and split the sample into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True)

#standarize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

dtc = tree.DecisionTreeClassifier(criterion='entropy')

#train model
dtc.fit(X_train, y_train)
score = dtc.score(X_test, y_test)

#print classification report
dtc = dtc.predict(X_test)
report = classification_report(y_test, dtc)
print(report)

print("Test score with L1 penalty: %.4f" % score)

              precision    recall  f1-score   support

      bottle       0.50      0.12      0.20         8
         cat       0.15      0.22      0.18         9
       chair       0.38      0.45      0.42        11
        face       0.78      0.54      0.64        13
       house       0.80      0.73      0.76        11
        rest       0.77      0.75      0.76        67
    scissors       0.56      0.42      0.48        12
scrambledpix       0.13      0.29      0.18         7
        shoe       0.20      0.25      0.22         8

    accuracy                           0.56       146
   macro avg       0.47      0.42      0.43       146
weighted avg       0.61      0.56      0.58       146

Test score with L1 penalty: 0.5616


### Neural Networks
Class MLPClassifier implements a multi-layer perceptron algorithm that trains using Backpropagation.

In [26]:
#select data
X = masker.fit_transform(func_file)
y = beh_label['labels']

#shuffle the data and split the sample into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True)

#standarize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#multinomial logistic regression object using L1 penalty
nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

MLPClassifier(activation='logistic', alpha=1e-05, batch_size='auto',
              beta_1=0.9, beta_2=0.999, early_stopping=False,
              epsilon=1e-08, hidden_layer_sizes=(5, 2),
              max_iter=200, momentum=0.9, n_iter_no_change=10,
              nesterovs_momentum=True, power_t=0.5, random_state=1,
              solver='lbfgs', tol=0.0001, validation_fraction=0.1, 
              verbose=False, warm_start=False)

#train model
nn.fit(X_train, y_train)
score = nn.score(X_test, y_test)

#print classification report
nn = nn.predict(X_test)
report = classification_report(y_test, nn)
print(report)

print("Test score with L1 penalty: %.4f" % score)

/home/srastegarnia/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


              precision    recall  f1-score   support

      bottle       0.25      0.50      0.33         6
         cat       0.38      0.38      0.38         8
       chair       0.33      0.50      0.40        10
        face       0.77      0.67      0.71        15
       house       0.42      0.45      0.43        11
        rest       0.89      0.77      0.82        73
    scissors       0.33      0.44      0.38         9
scrambledpix       0.67      0.33      0.44         6
        shoe       0.50      0.50      0.50         8

    accuracy                           0.63       146
   macro avg       0.50      0.50      0.49       146
weighted avg       0.68      0.63      0.65       146

Test score with L1 penalty: 0.6301


In [27]:
# # Prediction accuracy
# cv_scores_svc = cross_val_score(MLPClassifier, X_train, y_train, cv=5) 
# print(cv_scores)

# # The mean prediction accuracy
# classification_accuracy = np.mean(cv_scores_svc)
# classification_accuracy

In [36]:
# Run time for all these classifiers

# Make a data splitting object for cross validation
cv = LeaveOneGroupOut()

classifiers_scores = {}

for classifier_name, classifier in sorted(classifiers.items()):
    classifiers_scores[classifier_name] = {}
    print(70 * '_')

    for category in categories:
        classification_target = y[nonrest_task_mask].isin([category])
        t0 = time.time()
        classifiers_scores[classifier_name][category] = cross_val_score(
            classifier,
            masked_timecourses,
            classification_target,
            cv=cv,
            groups=session_labels,
            scoring="f1",
        )

        print(
            "%10s: %14s -- scores: %1.2f +- %1.2f, time %.2fs" %
            (
                classifier_name,
                category,
                classifiers_scores[classifier_name][category].mean(),
                classifiers_scores[classifier_name][category].std(),
                time.time() - t0,
            ),
        )

______________________________________________________________________


ValueError: Found input variables with inconsistent numbers of samples: [864, 864, 1452]

Prediction scores:

In [28]:
# A dictionary, to hold all our classifiers
classifiers = {'SVC': svm,
               'LogisticRegression': logistic_50,
               'KNeighborsClassifier':knn,
               'DecisionTreeClassifier': dtc,
               'MLPClassifier': nn,
               }

In [29]:
#select data
X = masker.fit_transform(func_file)
y = beh_label['labels']

#shuffle the data and split the sample into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True)

#standarize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# masked_timecourses = masker.fit_transform(func_file)[nonrest_task_mask]

In [30]:
# extract tags indicating to which acquisition run a tag belongs
session_labels = beh_label["chunks"]

In [33]:
# Run time for all these classifiers

# Make a data splitting object for cross validation
cv = LeaveOneGroupOut()

classifiers_scores = {}

for classifier_name, classifier in sorted(classifiers.items()):
    classifiers_scores[classifier_name] = {}
    print(70 * '_')

    for category in categories:
        classification_target = y.isin([category])
        t0 = time.time()
        classifiers_scores[classifier_name][category] = cross_val_score(
            classifier,
            X,
            y,
            cv=5,
            verbose=1
        )

        print(
            "%10s: %14s -- scores: %1.2f +- %1.2f, time %.2fs" %
            (
                classifier_name,
                category,
                classifiers_scores[classifier_name][category].mean(),
                classifiers_scores[classifier_name][category].std(),
                time.time() - t0,
            ),
        )

______________________________________________________________________


TypeError: estimator should be an estimator implementing 'fit' method, array(['scrambledpix', 'rest', 'rest', 'rest', 'rest', 'rest', 'scissors',
       'rest', 'scrambledpix', 'rest', 'rest', 'house', 'cat',
       'scrambledpix', 'scissors', 'rest', 'face', 'rest', 'scrambledpix',
       'cat', 'rest', 'rest', 'rest', 'cat', 'scissors', 'rest', 'rest',
       'rest', 'scrambledpix', 'shoe', 'face', 'cat', 'rest', 'scissors',
       'chair', 'rest', 'rest', 'rest', 'house', 'rest', 'scrambledpix',
       'chair', 'cat', 'shoe', 'house', 'shoe', 'scrambledpix', 'face',
       'rest', 'chair', 'rest', 'rest', 'rest', 'shoe', 'rest', 'chair',
       'scrambledpix', 'rest', 'cat', 'rest', 'rest', 'scissors', 'rest',
       'rest', 'rest', 'rest', 'rest', 'face', 'shoe', 'scrambledpix',
       'rest', 'rest', 'scrambledpix', 'rest', 'scissors', 'rest', 'rest',
       'face', 'rest', 'rest', 'rest', 'rest', 'scissors', 'cat', 'rest',
       'face', 'shoe', 'rest', 'chair', 'house', 'chair',

In [ ]:
# Make a rudimentary diagram

plt.figure()

tick_position = np.arange(len(categories))
plt.xticks(tick_position, categories, rotation=45)

for color, classifier_name in zip(
        [#'#E8D0A9', '#B7AFA3', '#771E10', 
         '#48110C', '#808080', '#DB4C2C', '#E38C2D', '#EBC137'],
        sorted(classifiers)):
    score_means = [classifiers_scores[classifier_name][category].mean()
                   for category in categories]
    plt.bar(tick_position, score_means, label=classifier_name, width=.11, color=color)
    tick_position = tick_position + .09

plt.ylabel('Classification accurancy (f1 score)')
plt.xlabel('Visual stimuli category')
plt.ylim(ymin=0)
plt.legend(bbox_to_anchor=(1, 1))
plt.title('Category-specific classification accuracy for different classifiers')
plt.tight_layout()

In [ ]:
# Plot the face vs house map for the different classifiers

mean_epi_img = image.mean_img(func_file)

# Restrict the decoding to face vs house
condition_mask = y.isin(['face', 'house'])
masked_timecourses = masked_timecourses[
    condition_mask[nonrest_task_mask]]
y_f = (y[condition_mask] == 'face')
# Transform the stimuli to binary values
y_f.astype(np.int)

for classifier_name, classifier in sorted(classifiers.items()):
    classifier.fit(masked_timecourses, y_f)

    if hasattr(classifier, 'coef_'):
        weights = classifier.coef_[0]
    elif hasattr(classifier, 'best_estimator_'):
        weights = classifier.best_estimator_.coef_[0]
    else:
        continue
    weight_img = masker.inverse_transform(weights)
    weight_map = get_data(weight_img)
    threshold = np.max(np.abs(weight_map)) * 1e-3
    plot_stat_map(weight_img, bg_img=mean_epi_img, display_mode='z', cut_coords=[-15],
                  threshold=threshold, title='%s: face vs house' % classifier_name)

In [ ]:
# # Shuffling
# X_train, X_test, y_train, y_test=train_test_split(func_masked, targets_masked, test_size=0.1, random_state=42, shuffle=True)

## Decoding with ANOVA + SVM: face vs house in the Haxby dataset

In [ ]:
# Restrict the analysis to faces and places
condition_mask = beh_label['labels'].isin(['face', 'house'])
conditions_f_h = y[condition_mask]

# Confirm that I now have 2 conditions
print(conditions_f_h.unique())

# Record these as an array of sessions, with fields
# for condition (face or house) and run
session_f_h = beh_label[condition_mask].to_records(index=False)
print(session_f_h.dtype.names)

In [ ]:
# Apply our condition_mask to fMRI data
X_f_h = X[condition_mask]

In [ ]:
#Build the decoder

#Define the prediction function to be used. Here I am using a Support Vector Classification, with a linear kernel
svc = SVC(kernel='linear')

# Define the dimension reduction to be used. (keep 5% of voxels)
feature_selection = SelectPercentile(f_classif, percentile=5)

# I have SVC classifier and our feature selection (SelectPercentile),then plug them together in a *pipeline*:
anova_svc = Pipeline([('anova', feature_selection), ('svc', svc)])

In [ ]:
# Fit the decoder and predict
anova_svc.fit(X_f_h, conditions_f_h)
y_pred = anova_svc.predict(X_f_h)

In [ ]:
# Obtain prediction scores via cross validation

# Define the cross-validation scheme used for validation, using LeaveOneGroupOut cross-validation.
cv = LeaveOneGroupOut()

# Compute the prediction accuracy for the different folds (i.e. session)
cv_scores = cross_val_score(anova_svc, X_f_h, conditions_f_h, cv=cv, groups=session_f_h)

# Return the corresponding mean prediction accuracy
classification_accuracy = cv_scores.mean()

# Print the results
print("Classification accuracy: %.4f / Chance level: %f" % (classification_accuracy, 1. / len(conditions_f_h.unique())))

Visualizing the results:

In [ ]:
# Look at the SVC’s discriminating weights
coef = svc.coef_
# reverse feature selection
coef = feature_selection.inverse_transform(coef)
# reverse masking
weight_img = masker.inverse_transform(coef)

# Use the mean image as a background to avoid relying on anatomical data
mean_img = image.mean_img(func_file)

# Create the figure
plot_stat_map(weight_img, mean_img, title='SVM weights')

# Save the results as a Nifti file
#weight_img.to_file('haxby_face_vs_house.nii')

## ROI-based decoding analysis
In this section, I am looking at decoding accuracy for different objects in three different masks: the full ventral stream (mask_vt), the house selective areas (mask_house), and the face-selective areas (mask_face), that have been defined via a standard General Linear Model (GLM) based analysis.

In [ ]:
# extract tags indicating to which acquisition run a tag belongs
session_labels = beh_label["chunks"][nonrest_task_mask]

In [ ]:
# The classifier: a support vector classifier
classifier = SVC(C=1., kernel="linear")

# A classifier to set the chance level
dummy_classifier = DummyClassifier()

# Make a data splitting object for cross validation
cv = LeaveOneGroupOut()

mask_names = ['mask_vt', 'mask_face', 'mask_house']

mask_scores = {}
mask_chance_scores = {}

for mask_name in mask_names:
    print("Working on mask %s" % mask_name)
    
    # Standardizing
    mask_filename = haxby_ds[mask_name][0]
    masker = NiftiMasker(mask_img=mask_filename, standardize=True)
    masked_timecourses = masker.fit_transform(func_file)[nonrest_task_mask]

    mask_scores[mask_name] = {}
    mask_chance_scores[mask_name] = {}

    for category in categories:
        print("Processing %s %s" % (mask_name, category))
        classification_target = (y[nonrest_task_mask] == category)
        mask_scores[mask_name][category] = cross_val_score(
            classifier,
            masked_timecourses,
            classification_target,
            cv=cv,
            groups=session_labels,
            scoring="roc_auc",
        )

        mask_chance_scores[mask_name][category] = cross_val_score(
            dummy_classifier,
            masked_timecourses,
            classification_target,
            cv=cv,
            groups=session_labels,
            scoring="roc_auc",
        )

        print("Scores: %1.2f +- %1.2f" % (
            mask_scores[mask_name][category].mean(),
            mask_scores[mask_name][category].std()))

## Different multi-class strategies
I compare one vs all and one vs one multi-class strategies: the overall cross-validated accuracy and the confusion matrix.

In [ ]:
# Build the decoders, using scikit-learn

svc_ovo = OneVsOneClassifier(Pipeline([
    ('anova', SelectKBest(f_classif, k=500)),
    ('svc', SVC(kernel='linear'))
]))

svc_ova = OneVsRestClassifier(Pipeline([
    ('anova', SelectKBest(f_classif, k=500)),
    ('svc', SVC(kernel='linear'))
]))

In [ ]:
# Remove the "rest" condition
y = y[nonrest_task_mask]
X = X[nonrest_task_mask]

cv_scores_ovo = cross_val_score(svc_ovo, X, y, cv=5, verbose=1)

cv_scores_ova = cross_val_score(svc_ova, X, y, cv=5, verbose=1)

print('OvO:', cv_scores_ovo.mean())
print('OvA:', cv_scores_ova.mean())

In [ ]:
# Plot barplots of the prediction scores

plt.figure(figsize=(4, 3))
plt.boxplot([cv_scores_ova, cv_scores_ovo])
plt.xticks([1, 2], ['One vs All', 'One vs One'])
plt.title('Prediction: accuracy score')

Plot a confusion matrix:

In [ ]:
# I fit on the the first 5 sessions and plot a confusion matrix on the last 2 sessions

svc_ovo.fit(X[session_labels < 5], y[session_labels < 5])
y_pred_ovo = svc_ovo.predict(X[session_labels >= 5])

plot_matrix(confusion_matrix(y_pred_ovo, y[session_labels >= 5]),
            labels=unique_conditions, cmap='hot_r')
plt.title('Confusion matrix: One vs One')
plt.xticks(rotation=45)
plt.yticks(rotation=0)

svc_ova.fit(X[session_labels < 5], y[session_labels < 5])
y_pred_ova = svc_ova.predict(X[session_labels >= 5])

plot_matrix(confusion_matrix(y_pred_ova, y[session_labels >= 5]),
            labels=unique_conditions, cmap='hot_r')
plt.title('Confusion matrix: One vs All')
plt.xticks(rotation=45)
plt.yticks(rotation=0)